## **<font color='#8d5383'>import library</font>**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import r2_score
import math

In [ ]:

# تابع برای نمونه‌گیری از CMP

def sample_cmp(lam, nu, size=1, max_y=100):
    samples = []
    for _ in range(size):
        log_probs = np.array([y * np.log(lam) - nu * math.lgamma(y + 1) for y in range(max_y+1)])
        log_probs -= log_probs.max()
        probs = np.exp(log_probs)
        probs /= probs.sum()
        y = np.random.choice(np.arange(max_y+1), p=probs)
        samples.append(y)
    return np.array(samples)


# تولید داده‌های شبیه‌سازی

np.random.seed(42)
n = 1000
p = 5

X = np.random.uniform(0, 1, size=(n, p))

eta = (
    0.5
    + 1.2 * X[:,0]
    - 0.8 * X[:,1]**2
    + 0.6 * np.sin(2*np.pi*X[:,2])
    + 0.5 * np.exp(X[:,3])
    - 0.7 * X[:,4]*X[:,0]
)

lam = np.exp(eta)

# شبیه‌سازی Y
Y_over = np.array([sample_cmp(l, nu=0.8, size=1)[0] for l in lam])
Y_under = np.array([sample_cmp(l, nu=2.0, size=1)[0] for l in lam])

Y_Poi = np.array([sample_cmp(l, nu=1, size=1)[0] for l in lam])


# نتایج آماری در جدول

results = pd.DataFrame({
    "سناریو": ["بیش‌پراکنش (ν=0.8)", "کم‌پراکنش (ν=2)", "عدم پراکندگی (ν=1)"],
    "میانگین": [np.mean(Y_over), np.mean(Y_under),np.mean(Y_Poi)],
    "واریانس": [np.var(Y_over), np.var(Y_under), np.var(Y_Poi)]
})

# گرد کردن به دو رقم اعشار
results = results.round(2)
results


,سناریو,میانگین,واریانس
0,بیش‌پراکنش (ν=0.8),8.91,63.25
1,کم‌پراکنش (ν=2),1.92,1.59
2,عدم پراکندگی (ν=1),5.31,17.54


## **<font color='#8d5383'>creare Models</font>**

In [ ]:
# Step 1: Implement the COM-Poisson loss function

class COM_Poisson_Loss(nn.Module):
    def __init__(self, max_iter=100):
        super(COM_Poisson_Loss, self).__init__()
        self.max_iter = max_iter

    def forward(self, y_pred, y_true):
        lambdaa = y_pred[:, 0]
        nu = y_pred[:, 1]
        y = y_true.float()

        S1 = y.sum(dim=0)
        S2 = torch.lgamma(y + 1).sum(dim=0)

        log_lambda = torch.log(torch.clamp(lambdaa, min=1e-6))
        Z_terms = []
        for j in range(self.max_iter):
            j_val = float(j)
            log_term = j_val * log_lambda - nu * torch.lgamma(torch.tensor(j_val + 1.0))
            Z_terms.append(log_term.unsqueeze(0))

        Z_stack = torch.cat(Z_terms, dim=0)
        logZ = torch.logsumexp(Z_stack, dim=0)

        log_likelihood = S1 * log_lambda - nu * S2 - y.size(0) * logZ
        loss = -log_likelihood.mean()
        return loss

In [ ]:
# Step 2: Define a simple neural network

class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, 2)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        lambdaa = torch.exp(torch.clamp(x[:, 0], min=-5, max=5))
        nu = torch.exp(torch.clamp(x[:, 1], min=-2, max=2))
        nu = torch.clamp(nu, min=0.5, max=10.0)
        return torch.stack([lambdaa, nu], dim=1)

In [ ]:
# Step 3: Train the model

def train_model(model, criterion, optimizer, X_train, y_train, epochs=100):
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        y_pred = model(X_train)
        loss = criterion(y_pred, y_train)
        if torch.isnan(loss):
            print("NaN in loss at epoch", epoch+1)
            return
        loss.backward()
        optimizer.step()
        if (epoch + 1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

In [ ]:
Y = Y_under

# تقسیم داده به آموزش و آزمون (80% آموزش، 20% آزمون)
X_train_np, X_test_np, y_train_np, y_test_np = train_test_split(X, Y, test_size=0.2, random_state=42)

# تبدیل به تنسورهای PyTorch
X_train = torch.tensor(X_train_np, dtype=torch.float32)
X_test = torch.tensor(X_test_np, dtype=torch.float32)
y_train = torch.tensor(y_train_np, dtype=torch.long)  # برای شمارش صحیح
y_test = torch.tensor(y_test_np, dtype=torch.long)

In [ ]:
def train_model(model, criterion, optimizer, X_train, y_train, X_test, y_test, epochs=200):
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    model.train()
    for epoch in range(epochs):
        total_loss = 0.0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            if torch.isnan(loss):
                print(f"NaN loss detected at epoch {epoch+1}")
                return
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        # نمایش هر 20 دوره
        if (epoch + 1) % 20 == 0:
            print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(train_loader):.4f}")

# ایجاد و آموزش مدل

input_size = X_train.shape[1]
model = SimpleNN(input_size, hidden_size=20)
criterion = COM_Poisson_Loss(max_iter=100)
optimizer = optim.Adam(model.parameters(), lr=0.01)

print("\nشروع آموزش مدل...")
train_model(model, criterion, optimizer, X_train, y_train, X_test, y_test, epochs=1000)


شروع آموزش مدل...
Epoch [20/1000], Loss: 51.1153
Epoch [40/1000], Loss: 51.2040
Epoch [60/1000], Loss: 51.0750
Epoch [80/1000], Loss: 51.0275
Epoch [100/1000], Loss: 51.1147
Epoch [120/1000], Loss: 51.0844
Epoch [140/1000], Loss: 51.0257
Epoch [160/1000], Loss: 51.0545
Epoch [180/1000], Loss: 51.0574
Epoch [200/1000], Loss: 51.0767
Epoch [220/1000], Loss: 51.0023
Epoch [240/1000], Loss: 51.0295
Epoch [260/1000], Loss: 51.0284
Epoch [280/1000], Loss: 51.0079
Epoch [300/1000], Loss: 51.0244
Epoch [320/1000], Loss: 51.0036
Epoch [340/1000], Loss: 51.0223
Epoch [360/1000], Loss: 50.9972
Epoch [380/1000], Loss: 51.0272
Epoch [400/1000], Loss: 51.0119
Epoch [420/1000], Loss: 51.0655
Epoch [440/1000], Loss: 51.0847
Epoch [460/1000], Loss: 51.0433
Epoch [480/1000], Loss: 50.9990
Epoch [500/1000], Loss: 50.9979
Epoch [520/1000], Loss: 51.0581
Epoch [540/1000], Loss: 51.0190
Epoch [560/1000], Loss: 51.0115
Epoch [580/1000], Loss: 51.0651
Epoch [600/1000], Loss: 51.0578
Epoch [620/1000], Loss: 5

In [ ]:

# محاسبه MAE، RMSE، MSE، AIC، BIC

def compute_metrics(model, criterion, X_test, y_test):
    model.eval()
    with torch.no_grad():
        y_pred_test = model(X_test)  # خروجی: [lambda_pred, nu_pred]
        lambda_pred = y_pred_test[:, 0].cpu().numpy()  # تبدیل به numpy
        nu_pred = y_pred_test[:, 1].cpu().numpy()
        y_true = y_test.cpu().numpy()

    # 1. MAE، MSE، RMSE نسبت به مقادیر واقعی Y
    mae = np.mean(np.abs(y_true - lambda_pred))
    mse = np.mean((y_true - lambda_pred) ** 2)
    rmse = np.sqrt(mse)

    # 2. محاسبه لگاریتم درست‌نمایی کل (log-likelihood)
    def cmp_log_likelihood_single(y, lam, nu, max_y=100):
        # محاسبه log(P(Y=y | λ, ν))
        log_probs = np.array([k * np.log(lam) - nu * math.lgamma(k + 1) for k in range(max_y + 1)])
        log_probs -= log_probs.max()  # پایداری عددی
        probs = np.exp(log_probs)
        probs /= probs.sum()
        y_int = int(y)
        if y_int < len(probs):
            return np.log(probs[y_int] + 1e-12)  # جلوگیری از log(0)
        else:
            return -np.inf

    # محاسبه log-likelihood برای تمام نمونه‌های آزمون
    log_likelihood = 0.0
    for i in range(len(y_true)):
        ll = cmp_log_likelihood_single(
            y=y_true[i],
            lam=lambda_pred[i],
            nu=nu_pred[i],
            max_y=100
        )
        log_likelihood += ll

    n = len(y_true)
    k = sum(p.numel() for p in model.parameters())  # تعداد کل پارامترها

    # 3. AIC و BIC
    aic = -2 * log_likelihood + 2 * k
    bic = -2 * log_likelihood + k * np.log(n)

    return {
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'LogLikelihood': log_likelihood,
        'AIC': aic,
        'BIC': bic,
        'N': n,
        'Num_Params': k
    }

# اجرای محاسبات

metrics = compute_metrics(model, criterion, X_test, y_test)

print("\n📊 نتایج ارزیابی مدل کام-پواسن (روی داده آزمون):")
print(f"MAE:      {metrics['MAE']:.4f}")
print(f"MSE:      {metrics['MSE']:.4f}")
print(f"RMSE:     {metrics['RMSE']:.4f}")
print(f"Log-Lik:  {metrics['LogLikelihood']:.4f}")
print(f"AIC:      {metrics['AIC']:.4f}")
print(f"BIC:      {metrics['BIC']:.4f}")
print(f"تعداد پارامترها: {metrics['Num_Params']}")
print(f"تعداد نمونه‌ها:   {metrics['N']}")


📊 نتایج ارزیابی مدل کام-پواسن (روی داده آزمون):
MAE:      1.2671
MSE:      2.2291
RMSE:     1.4930
Log-Lik:  -319.3372
AIC:      962.6744
BIC:      1497.0018
تعداد پارامترها: 162
تعداد نمونه‌ها:   200


## رگرسیون پواسون شبکه عصبی

In [ ]:

Y = Y_Poi

# تقسیم داده به آموزش و آزمون (80% آموزش، 20% آزمون)
X_train_np, X_test_np, y_train_np, y_test_np = train_test_split(X, Y, test_size=0.2, random_state=42)

# تبدیل به تنسورهای PyTorch
X_train = torch.tensor(X_train_np, dtype=torch.float32)
X_test = torch.tensor(X_test_np, dtype=torch.float32)
y_train = torch.tensor(y_train_np, dtype=torch.long)  # برای شمارش صحیح
y_test = torch.tensor(y_test_np, dtype=torch.long)

In [ ]:
class PoissonNN(nn.Module):
    def __init__(self, input_size, hidden_size=20):
        super(PoissonNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, 1)  # فقط یک خروجی: λ

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        lambda_pred = torch.exp(x).squeeze()  # exp برای مثبت‌بودن + squeeze برای شکل (batch,)
        return lambda_pred

In [ ]:
def train_model(model, criterion, optimizer, X_train, y_train, X_test, y_test, epochs=200):
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    model.train()
    for epoch in range(epochs):
        total_loss = 0.0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            if torch.isnan(loss):
                print(f"NaN loss detected at epoch {epoch+1}")
                return
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        # نمایش هر 20 دوره
        if (epoch + 1) % 20 == 0:
            print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(train_loader):.4f}")

# -----------------------------
# ایجاد و آموزش مدل
# -----------------------------
input_size = X_train.shape[1]
model = PoissonNN(input_size, hidden_size=20)
criterion = criterion = nn.PoissonNLLLoss(log_input=True, full=False, eps=1e-8)
optimizer = optim.Adam(model.parameters(), lr=0.01)

print("\nشروع آموزش مدل...")
train_model(model, criterion, optimizer, X_train, y_train, X_test, y_test, epochs=1000)


شروع آموزش مدل...
Epoch [20/1000], Loss: -4.2878
Epoch [40/1000], Loss: -4.3410
Epoch [60/1000], Loss: -4.3473
Epoch [80/1000], Loss: -4.3194
Epoch [100/1000], Loss: -4.3680
Epoch [120/1000], Loss: -4.3822
Epoch [140/1000], Loss: -4.3765
Epoch [160/1000], Loss: -4.3596
Epoch [180/1000], Loss: -4.3461
Epoch [200/1000], Loss: -4.3842
Epoch [220/1000], Loss: -4.3867
Epoch [240/1000], Loss: -4.4217
Epoch [260/1000], Loss: -4.4333
Epoch [280/1000], Loss: -4.4448
Epoch [300/1000], Loss: -4.4263
Epoch [320/1000], Loss: -4.4400
Epoch [340/1000], Loss: -4.4578
Epoch [360/1000], Loss: -4.4474
Epoch [380/1000], Loss: -4.4461
Epoch [400/1000], Loss: -4.4549
Epoch [420/1000], Loss: -4.4572
Epoch [440/1000], Loss: -4.4566
Epoch [460/1000], Loss: -4.4588
Epoch [480/1000], Loss: -4.4525
Epoch [500/1000], Loss: -4.4424
Epoch [520/1000], Loss: -4.4389
Epoch [540/1000], Loss: -4.4462
Epoch [560/1000], Loss: -4.4456
Epoch [580/1000], Loss: -4.4422
Epoch [600/1000], Loss: -4.4624
Epoch [620/1000], Loss: -

In [ ]:

# محاسبه معیارهای ارزیابی

def compute_poisson_metrics(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        lambda_pred = model(X_test).cpu().numpy()
        y_true = y_test.cpu().numpy()

    # 1. MAE، MSE، RMSE
    mae = np.mean(np.abs(y_true - lambda_pred))
    mse = np.mean((y_true - lambda_pred) ** 2)
    rmse = np.sqrt(mse)

    # 2. Log-Likelihood (پواسون)
    log_likelihood = np.sum(y_true * np.log(lambda_pred + 1e-8) - lambda_pred)

    # 3. AIC و BIC
    n = len(y_true)
    k = sum(p.numel() for p in model.parameters())  # تعداد پارامترها

    #aic = -2 * log_likelihood + 2 * k
    #bic = -2 * log_likelihood + k * np.log(n)

    aic = 2 * k + (-2) * log_likelihood
    bic = k * np.log(n) + (-2) * log_likelihood

    return {
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'LogLikelihood': log_likelihood,
        'AIC': aic,
        'BIC': bic,
        'N': n,
        'Num_Params': k
    }

# اجرای ارزیابی

metrics_poisson = compute_poisson_metrics(model, X_test, y_test)

print("\n📊 نتایج ارزیابی مدل پواسون (روی داده آزمون):")
print(f"MAE:      {metrics_poisson['MAE']:.4f}")
print(f"MSE:      {metrics_poisson['MSE']:.4f}")
print(f"RMSE:     {metrics_poisson['RMSE']:.4f}")
print(f"Log-Lik:  {metrics_poisson['LogLikelihood']:.4f}")
print(f"AIC:      {metrics_poisson['AIC']:.4f}")
print(f"BIC:      {metrics_poisson['BIC']:.4f}")
print(f"تعداد پارامترها: {metrics_poisson['Num_Params']}")
print(f"تعداد نمونه‌ها:   {metrics_poisson['N']}")


📊 نتایج ارزیابی مدل پواسون (روی داده آزمون):
MAE:      4.1448
MSE:      31.7851
RMSE:     5.6378
Log-Lik:  360.4607
AIC:      -438.9215
BIC:      26.1413
تعداد پارامترها: 141
تعداد نمونه‌ها:   200
